In [1]:
try:
    import pandas as pd
    import time as time
    import database as db
    from bs4 import BeautifulSoup
    from unidecode import unidecode
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys 
except Exception as e:
    print(e)

In [2]:
data = db.DataBase()

In [85]:
def getData(ano, formacao, inicio=0, limitePaginas=None, curriculo=1):
       
    navegador = webdriver.Chrome()
    navegador.get("http://buscatextual.cnpq.br/buscatextual/busca.do")

    #Tags de pesquisa
    if formacao != "doutorado":
        navegador.find_element_by_xpath('//*[@id="buscarDemais"]').send_keys(Keys.SPACE) #Buscar demais pesquisadores

        if formacao == "mestrado":
            navegador.find_element_by_xpath('//*[@id="filtro2"]').click()
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').click()
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
            navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.ENTER)
            time.sleep(1)
            navegador.find_element_by_xpath('//*[@id="preencheCategoriaNivelBolsa"]')

    navegador.find_element_by_xpath('//*[@id="buscarEstrangeiros"]').click()
            
    navegador.find_element_by_xpath('//*[@id="buscaAssunto"]').send_keys(Keys.SPACE) #Buscar por assunto

    #Caixa de texto
    buscar = navegador.find_element_by_xpath('//*[@id="textoBusca"]')
    buscar.send_keys(ano)
    buscar.send_keys(Keys.ENTER)

    time.sleep(2)

    passadas = []

    janelaAtual = navegador.current_window_handle
    passadas.append(navegador.window_handles)

    #Variaveis de controle da paginação
    pagina = 1
    pag = 2
    troca = True
    
    while(True):
        #Troca de pagina
        if inicio!=0:
            navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
            pagina = inicio
            link = navegador.current_url.split(";")
            link[0] = link[0].replace(str(10), str((pagina-1)*10))
            link = link[0]+";"+link[1]
            
            pag = inicio+1
            if inicio<11:
                pag = inicio+1
            elif inicio%10==0:
                pag = 13
            else:
                div = "1"+("0"*(len(str(inicio))-1))
                pag = int((inicio+1)/int(div))+2
            navegador.get(link)
            troca = False
            inicio = 0
        
        if troca:
            try:
                if pagina!=1 and pag<=11 and pagina<=11:
                    navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                    pag = pag+1 if pag<11 else 3

                if pagina>11 and pag>2:
                    navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                    pag = pag+1 if pag<13 else 3
            except Exception as e:
                print(e)
                break

        #Limitador de pagínas
        if limitePaginas != None and pagina == limitePaginas:
            break

        time.sleep(5)
        
        troca = True
        
        for i in range(curriculo,11):
            try:
                #Selecionar curriculo
                navegador.find_element_by_xpath(f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a').click()

                time.sleep(5)

                #Abrir curriculo
                navegador.find_element_by_xpath('//*[@id="idbtnabrircurriculo"]').click()
            
            except:
                i -= 1
                navegador.get(navegador.current_url)
                
            time.sleep(5)

            try:
                for janela in navegador.window_handles:
                    if janela not in passadas:
                        navegador.switch_to.window(janela)

            except:
                navegador.switch_to.window(janelaAtual)
                navegador.get(navegador.current_url)
                i -= 1
                continue
            
            try:
                #Pegar dados 
                site = BeautifulSoup(navegador.page_source, 'html.parser')
                artigos = []

                for artigo in site.findAll('div', attrs={'class':['artigo-completo']}):
                    if "2021" in artigo.text:
                        artigo = artigo.text[10:-4:]
                        text = ""
                        maior = 0
                        for s in artigo.split("."):
                            for ss in s.split(","):
                                if len(ss) > maior:
                                    text = ss
                                    maior = len(ss)
                        artigos.append(text)

                nome = site.findAll('h2', attrs={'class':['nome']})[0].text
            except:
                navegador.close()
                time.sleep(1)
                #navegador.switch_to.window(janelaAtual)
                i-=1
                continue
            
            #Inserir no banco de dados
            if artigos != []:
                data.insertData(nome, formacao, artigos, ano)

            navegador.close()

            time.sleep(1)

            navegador.switch_to.window(janelaAtual)

            #Fechar curriculo
            navegador.find_element_by_xpath('//*[@id="idbtnfechar"]').click()

        data.connection.commit()
        pagina += 1
        curriculo = 1
        
    navegador.quit()
    return pagina, i

In [ ]:
pagina, index = getData("2018", "doutorado", inicio=99, curriculo=10)

#getData("2019", "doutorado", 100)

In [90]:
data.selectData(2018)

,primeiro_nome,nome_completo,formacao,titulo,ano


In [91]:
data.selectData(2019)

,primeiro_nome,nome_completo,formacao,titulo,ano
